In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch

In [2]:
df = pd.read_csv("../data\SUTT Seniors Test - Recruitments 2024 - SUTT Excomm.csv")

df_original = df.copy()
df

df = df.rename(columns={" AGES":"AGES", "CGPA-2":"CGPA - 2", " HAS A S/O ?": "HAS A S/O ?", " CLUB / DEPARTMENT": "CLUB / DEPARTMENT"})

df

,NAME,SEX,AGES,ADMISSION MODE,CGPA - 1,CGPA - 2,CLUB / DEPARTMENT,BOARD STUDIED,DAYS SINCE LAST SHOWER,BIRTHDAY,TREATS GIVEN,HAS A S/O ?,ATTENDS LECTURES,CHARACTER,PIZZERIA EXPENDITURE (annual),GRAMS / MONTH
0,Vaasu,female,18,BOARD TOPPER,3.24,4,SUTT,CBSE,14,9/15/2006,0,NO,YES,GHOT,20898,2.5
1,Rajat,male,19,BITSAT,6.45,6,SUTT,CBSE,8,7/23/2005,1,YES,YES,LITE,11695,5.7
2,Akhil,male,16,BITSAT,6.23,7,SUTT,CBSE,10,4/10/2007,3,NO,NO,LITE,16116,22.0
3,Yash,male,17,BITSAT,8.43,10,SUTT,CBSE,4,7/18/2007,4,NO,YES,LITE,13938,0.8
4,Yashwardhan,male,17,BITSAT,3.76,6,SUTT,CBSE,13,7/13/2007,3,YES,YES,LITE,18635,5.4
5,Aaryan,male,17,BITSAT,8.20,9,SUTT,CBSE,5,8/24/2007,4,YES,YES,LITE,19564,2.5
6,Siddharth,male,19,BITSAT,4.01,7,SUTT,CBSE,7,2/17/2005,4,NO,YES,GHOT,13060,0.6
7,Hemant,male,17,BITSAT,7.16,4,SUTT,CBSE,3,10/9/2007,2,YES,YES,LITE,11381,0.9
8,Manish,male,18,BITSAT,9.03,6,SUTT,CBSE,9,3/26/2006,2,FALSE,YES,LITE,17796,3.7
9,Tanay,male,19,BITSAT,5.73,4,SUTT,CBSE,8,2/11/2005,3,NO,NO,LITE,13179,7.8


### Encoding

In [3]:
admissionConverted = pd.get_dummies(df['ADMISSION MODE'], drop_first=True)
boardConverted = pd.get_dummies(df['BOARD STUDIED'], drop_first=True)
characterConverted = pd.get_dummies(df['CHARACTER'], drop_first=True)
sexConverted = pd.get_dummies(df['SEX'], drop_first=True)
clubConverted = pd.get_dummies(df['CLUB / DEPARTMENT'])

df = pd.concat([sexConverted, df['AGES'], admissionConverted, df['CGPA - 1'], df['CGPA - 2'], 
           clubConverted, boardConverted, df['DAYS SINCE LAST SHOWER'], df['TREATS GIVEN'], 
           df['HAS A S/O ?'], df['ATTENDS LECTURES'], characterConverted, df['PIZZERIA EXPENDITURE (annual)'], 
           df['GRAMS / MONTH']], axis=1)

df ['HAS A S/O ?'].unique()
df.loc[df['HAS A S/O ?']=='YES', 'HAS A S/O ?'] = True
df.loc[df['HAS A S/O ?']=='FALSE', 'HAS A S/O ?'] = False
df.loc[df['HAS A S/O ?']=='NO', 'HAS A S/O ?'] = False

df['ATTENDS LECTURES'].unique()
df.loc[df['ATTENDS LECTURES']=='YES', 'ATTENDS LECTURES'] = True
df.loc[df['ATTENDS LECTURES']=='NO', 'ATTENDS LECTURES'] = False
for col in list(df.columns):
    df[col] = df[col].astype('float32')

df['None'] = 0
df['GPL'] = 0
df['DASA'] = 0
df['DVM'] = 0
df['Gaming Club'] = 0
df['Mountaineering'] = 0
df['ICSE'] = 0
df['Other'] = 0
df = df[list(('male', 'AGES', 'BOARD TOPPER', 'DASA', 'CGPA - 1', 'CGPA - 2', 'DVM', 'Gaming Club', 'Mountaineering', 'None', 'ICSE', 'Other', 'DAYS SINCE LAST SHOWER', 'TREATS GIVEN', 'HAS A S/O ?', 'ATTENDS LECTURES', 'LITE', 'PIZZERIA EXPENDITURE (annual)', 'GRAMS / MONTH', 'GPL'))]
df

,male,AGES,BOARD TOPPER,DASA,CGPA - 1,CGPA - 2,DVM,Gaming Club,Mountaineering,None,ICSE,Other,DAYS SINCE LAST SHOWER,TREATS GIVEN,HAS A S/O ?,ATTENDS LECTURES,LITE,PIZZERIA EXPENDITURE (annual),GRAMS / MONTH,GPL
0,0.0,18.0,1.0,0,3.24,4.0,0,0,0,0,0,0,14.0,0.0,0.0,1.0,0.0,20898.0,2.5,0
1,1.0,19.0,0.0,0,6.45,6.0,0,0,0,0,0,0,8.0,1.0,1.0,1.0,1.0,11695.0,5.7,0
2,1.0,16.0,0.0,0,6.23,7.0,0,0,0,0,0,0,10.0,3.0,0.0,0.0,1.0,16116.0,22.0,0
3,1.0,17.0,0.0,0,8.43,10.0,0,0,0,0,0,0,4.0,4.0,0.0,1.0,1.0,13938.0,0.8,0
4,1.0,17.0,0.0,0,3.76,6.0,0,0,0,0,0,0,13.0,3.0,1.0,1.0,1.0,18635.0,5.4,0
5,1.0,17.0,0.0,0,8.20,9.0,0,0,0,0,0,0,5.0,4.0,1.0,1.0,1.0,19564.0,2.5,0
6,1.0,19.0,0.0,0,4.01,7.0,0,0,0,0,0,0,7.0,4.0,0.0,1.0,0.0,13060.0,0.6,0
7,1.0,17.0,0.0,0,7.16,4.0,0,0,0,0,0,0,3.0,2.0,1.0,1.0,1.0,11381.0,0.9,0
8,1.0,18.0,0.0,0,9.03,6.0,0,0,0,0,0,0,9.0,2.0,0.0,1.0,1.0,17796.0,3.7,0
9,1.0,19.0,0.0,0,5.73,4.0,0,0,0,0,0,0,8.0,3.0,0.0,0.0,1.0,13179.0,7.8,0


In [4]:
import joblib

scaler = joblib.load('../data/scaler.save')
scaler

cols = list(df.columns)
df = scaler.transform(df)
df = pd.DataFrame(df)
df.columns = cols

df = df[['male', 'DASA', 'DVM', 'ATTENDS LECTURES', 'LITE','GRAMS / MONTH']]
in_features = len(list(df.columns))
cols = df.columns

In [5]:
df

,male,DASA,DVM,ATTENDS LECTURES,LITE,GRAMS / MONTH
0,0.0,0.0,0.0,1.0,0.0,0.216590
1,1.0,0.0,0.0,1.0,1.0,0.511521
2,1.0,0.0,0.0,0.0,1.0,2.013825
3,1.0,0.0,0.0,1.0,1.0,0.059908
4,1.0,0.0,0.0,1.0,1.0,0.483871
5,1.0,0.0,0.0,1.0,1.0,0.216590
6,1.0,0.0,0.0,1.0,0.0,0.041475
7,1.0,0.0,0.0,1.0,1.0,0.069124
8,1.0,0.0,0.0,1.0,1.0,0.327189
9,1.0,0.0,0.0,0.0,1.0,0.705069


### Predicting

In [6]:
from torch import nn
hidden_nodes = 2

class ModelV3_2(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.linear_1 = nn.Linear(in_features=in_features, out_features=hidden_nodes)
        self.linear_2 = nn.Linear(in_features=hidden_nodes, out_features=hidden_nodes)
        self.linear_3 = nn.Linear(in_features=hidden_nodes, out_features=hidden_nodes)
        self.linear_4 = nn.Linear(in_features=hidden_nodes, out_features=1)
    
    def forward(self, x):
        return self.linear_4(self.linear_3(self.linear_2(self.linear_1(x))))
    
model = ModelV3_2()
model.load_state_dict(torch.load("../models/model_3_2.pth"))
model

ModelV3_2(
  (linear_1): Linear(in_features=6, out_features=2, bias=True)
  (linear_2): Linear(in_features=2, out_features=2, bias=True)
  (linear_3): Linear(in_features=2, out_features=2, bias=True)
  (linear_4): Linear(in_features=2, out_features=1, bias=True)
)

In [7]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [8]:
model = model.to(device)
input_tensor = torch.Tensor(df.values)
input_tensor = input_tensor.to(device)

model.eval()
with torch.inference_mode():
    pred_logits = model(input_tensor).squeeze()
    pred_labels = torch.round(torch.sigmoid(pred_logits))
    pred_labels

model = model.to("cpu")
input_tensor = input_tensor.to("cpu")

In [9]:
pred_df = pd.DataFrame(pred_labels.detach().numpy())
pred_df.columns = ['GPL']
pred_df['GPL'] = pred_df['GPL'].astype('object')
pred_df.loc[pred_df['GPL']==0, 'GPL'] = 'NO'
pred_df.loc[pred_df['GPL']==1, 'GPL'] = 'YES'
pred_df

,GPL
0,NO
1,NO
2,NO
3,NO
4,NO
5,NO
6,YES
7,NO
8,NO
9,NO


In [10]:
df_original['GPL'] = pred_df['GPL']
df_original

,NAME,SEX,AGES,ADMISSION MODE,CGPA - 1,CGPA-2,CLUB / DEPARTMENT,BOARD STUDIED,DAYS SINCE LAST SHOWER,BIRTHDAY,TREATS GIVEN,HAS A S/O ?,ATTENDS LECTURES,CHARACTER,PIZZERIA EXPENDITURE (annual),GRAMS / MONTH,GPL
0,Vaasu,female,18,BOARD TOPPER,3.24,4,SUTT,CBSE,14,9/15/2006,0,NO,YES,GHOT,20898,2.5,NO
1,Rajat,male,19,BITSAT,6.45,6,SUTT,CBSE,8,7/23/2005,1,YES,YES,LITE,11695,5.7,NO
2,Akhil,male,16,BITSAT,6.23,7,SUTT,CBSE,10,4/10/2007,3,NO,NO,LITE,16116,22.0,NO
3,Yash,male,17,BITSAT,8.43,10,SUTT,CBSE,4,7/18/2007,4,NO,YES,LITE,13938,0.8,NO
4,Yashwardhan,male,17,BITSAT,3.76,6,SUTT,CBSE,13,7/13/2007,3,YES,YES,LITE,18635,5.4,NO
5,Aaryan,male,17,BITSAT,8.20,9,SUTT,CBSE,5,8/24/2007,4,YES,YES,LITE,19564,2.5,NO
6,Siddharth,male,19,BITSAT,4.01,7,SUTT,CBSE,7,2/17/2005,4,NO,YES,GHOT,13060,0.6,YES
7,Hemant,male,17,BITSAT,7.16,4,SUTT,CBSE,3,10/9/2007,2,YES,YES,LITE,11381,0.9,NO
8,Manish,male,18,BITSAT,9.03,6,SUTT,CBSE,9,3/26/2006,2,FALSE,YES,LITE,17796,3.7,NO
9,Tanay,male,19,BITSAT,5.73,4,SUTT,CBSE,8,2/11/2005,3,NO,NO,LITE,13179,7.8,NO


In [11]:
df_original.to_csv("../5_results/senior_predictions.csv", encoding='utf-8', index=False)